In [75]:
'''! pip install transformers
! pip install konlpy
! pip install pytorch_lightning'''

'! pip install transformers\n! pip install konlpy\n! pip install pytorch_lightning'

In [76]:
import torch
import os 
import sys
import pandas as pd
import numpy as np
from tqdm import tqdm

In [77]:
# seed
seed = 7777
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla P100-PCIE-16GB
cuda


In [78]:
df = pd.read_csv('/content/drive/MyDrive/sports_news_KoBART.csv')
df = df[['TITLE', 'CONTENT', 'PUBLISH_DT', 'KoBART']]
df

,TITLE,CONTENT,PUBLISH_DT,KoBART
0,스털링 다이빙 논란 종결 오른쪽 다리 접촉 있었잖아,유럽축구연맹 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크전에 나온 ...,2021-07-15,유럽축구연맹 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크전에 나온 ...
1,디 마리아 없다 유로X코파 베스트11 이탈리아만 7명,지난달 시작된 유로와 코파 아메리카가 11일을 끝으로 막을 내렸다 이탈리아는 결승전...,2021-07-15,지난달 시작된 시작된 유로와 코파 아메리카가 11일을 끝으로 막을 내렸다
2,슈퍼컴퓨터 예측 맨시티 우승맨유 4위 토트넘은 6위,새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다 영국 매체 스포츠 바이...,2021-07-15,지난 시즌 초반 고초를 겪었던 맨체스터 시티가 승점 88을 쌓아 경쟁 팀들을 손쉽게...
3,이재성 완벽한 프로 마인츠서 성공할 것 킬 디렉터의 애정 듬뿍 응원,홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다 이재성은 20...,2021-07-15,홀슈타인 킬 우베 스토버 디렉터가 킬의 에이스 이재성을 향해 응원 메시지를 띄웠다
4,홈킷과 딴판 바르사 팬들 NEW 어웨이 셔츠 호평 가장 좋아하는 색,FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다 팬들은 만족스럽다는 반응이다 바르...,2021-07-15,바르사는 15일 공식 홈페이지를 통해 20212022시즌에 입을 어웨이 킷을 선보였...
...,...,...,...,...
7325,이제 홈팬 야유 받는 먹튀 선수 차비 감독 조차 그만 해라,FC바르셀로나 팬들에게 우스망 뎀벨레는 밉상이 되어버렸다 바르사는 7일 오전 0시 ...,2022-02-07,7일 오전 0시 15분 스페인 바르셀로나 캄노우에서 열린 아틀레티코 마드리드와 20...
7326,오피셜 성남 만 17세 유스 김지수와 준프로 계약,성남FC가 만17세 2004년생 수비수 김지수와 준프로 계약을 체결했다 김지수는 1...,2022-02-07,성남FC가 만17세 2004년생 2004년생 수비수 김지수와 준프로 계약을 체결했다
7327,오베르마스 아약스서 쫓겨난다여성 동료들에게 부적절한 메시지,레전드 마르크 오베르마스가 아약스에서 쫓겨났다 이유는 굉장히 굴욕적이었다 아약스는 ...,2022-02-07,네덜란드 출신의 레전드 마르크 오베르마스가 아약스에서 쫓겨났다 이유는 굉장히 굴욕적이었다
7328,바르사 차비 감독 트라오레 데뷔전 활약에 깜짝몸이 야수 같아,FC 바르셀로나 차비 에르난데스 감독이 데뷔전을 치른 아다마 트라오레를 극찬했다 바...,2022-02-07,지난겨울 이적시장에서 울버햄프턴을 떠나 바르사로 임대 이적했고 이날 경기에 선발 출...


In [79]:
# 한국어 불용어 리스트 크롤링
import requests
from bs4 import BeautifulSoup

url = "https://www.ranks.nl/stopwords/korean"
response = requests.get(url, verify = False)

if response.status_code == 200:
    soup = BeautifulSoup(response.text,'html.parser')
    content = soup.select_one('#article178ebefbfb1b165454ec9f168f545239 > div.panel-body > table > tbody > tr')
    stop_words=[]
    for x in content.strings:
        x=x.strip()
        if x:
            stop_words.append(x)
    print(f"# Korean stop words: {len(stop_words)}")
else:
    print(response.status_code)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


# Korean stop words: 677


In [80]:
from konlpy.tag import Okt
okt = Okt()
for i in tqdm(range(len(df))):
  temp_data = okt.morphs(df["CONTENT"].iloc[i])
  temp_list = []
  
  for word in temp_data:
    if word in stop_words: continue
    temp_list.append(word)
  
  df["CONTENT"].iloc[i] = " ".join(temp_list)
  
  temp_data = okt.morphs(df["TITLE"].iloc[i])
  temp_list = []

  for word in temp_data:
    if word in stop_words: continue
    temp_list.append(word)
  
  df["TITLE"].iloc[i] = " ".join(temp_list)

  temp_data = okt.morphs(df["KoBART"].iloc[i])
  temp_list = []

  for word in temp_data:
    if word in stop_words: continue
    temp_list.append(word)
  
  df["KoBART"].iloc[i] = " ".join(temp_list)

100%|██████████| 7330/7330 [06:50<00:00, 17.87it/s]


In [81]:
df.head()

,TITLE,CONTENT,PUBLISH_DT,KoBART
0,스털링 다이빙 논란 종결 오른쪽 다리 접촉 있었잖아,유럽 축구 연맹 유로 2020 심판 위원장 로베르토 로세티 잉글랜드 덴마크 전 나온...,2021-07-15,유럽 축구 연맹 유로 2020 심판 위원장 로베르토 로세티 잉글랜드 덴마크 전 나온...
1,디 마리아 없다 유로 X 코파 베스트 11 이탈리아 만 7 명,지난달 시작 된 유로 코파 아메리카 11일 끝 막 내렸다 이탈리아 는 결승전 잉글랜...,2021-07-15,지난달 시작 된 시작 된 유로 코파 아메리카 11일 끝 막 내렸다
2,슈퍼컴퓨터 예측 맨시티 우승 맨유 4 위 토트넘 은 6 위,새 시즌 시작 하기도 전 슈퍼컴퓨터 예상 한 순위 나왔다 영국 매체 스포츠 바이블 ...,2021-07-15,지난 시즌 초반 고초 겪었던 맨체스터 시티 승점 88 쌓아 경쟁 팀 손쉽게 따돌리고...
3,이재성 완벽한 프로 마인츠 서 성공할 킬 디렉터 애정 듬뿍 응원,홀슈타인 킬 우베 스토 버 디렉터 이재성 향 해 응원 메시지 띄웠다 이재성 은 20...,2021-07-15,홀슈타인 킬 우베 스토 버 디렉터 킬 에이스 이재성 향 해 응원 메시지 띄웠다
4,홈킷 딴판 바르사 팬 NEW 웨이 셔츠 호평 가장 좋아하는 색,FC 바르셀로나 새 시즌 원정 유니폼 공개 했다 팬 은 만족스럽다는 반응 이다 바르...,2021-07-15,바르사 는 15일 공식 홈페이지 통해 20212022시 즌에 입 웨이 킷 선보였는데...


In [82]:
import os
import glob
import torch
import ast
import numpy as np
from tqdm import tqdm, trange
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler, random_split

In [83]:
! git clone https://github.com/seujung/KoGPT2-summarization

Cloning into 'KoGPT2-summarization'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 65 (delta 33), reused 46 (delta 18), pack-reused 0
Unpacking objects: 100% (65/65), done.


In [84]:
cd /content/KoGPT2_summarization

/content/KoGPT2_summarization


In [85]:
! pip install -r requirements.txt

In [86]:
from train_ptuning import *
from soft_embedding import *

In [87]:
cd data

/content/KoGPT2_summarization/data


In [88]:
df.to_csv('train.tsv', sep = '\t')

In [89]:
cd /content/KoGPT2_summarization

/content/KoGPT2_summarization


In [90]:
# 모델 환경 설정
parser = argparse.ArgumentParser(description='KoGPT2 Summarization')
parser = Base.add_model_specific_args(parser)
parser = ArgsBase.add_model_specific_args(parser)
parser = KoGPTSummaryModule.add_model_specific_args(parser)
parser = pl.Trainer.add_argparse_args(parser)
parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")
args = parser.parse_args()

In [91]:
# 로거 설정
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info(args)

INFO:root:Namespace(accelerator=None, accumulate_grad_batches=None, amp_backend='native', amp_level=None, auto_lr_find=False, auto_scale_batch_size=False, auto_select_gpus=False, batch_size=14, benchmark=False, check_val_every_n_epoch=1, checkpoint_callback=None, default_root_dir=None, detect_anomaly=False, deterministic=False, devices=None, enable_checkpointing=True, enable_model_summary=True, enable_progress_bar=True, fast_dev_run=False, fff='/root/.local/share/jupyter/runtime/kernel-41e96303-fb4f-4d8c-a912-3253c5be7021.json', flush_logs_every_n_steps=None, gpus=None, gradient_clip_algorithm=None, gradient_clip_val=None, ipus=None, limit_predict_batches=1.0, limit_test_batches=1.0, limit_train_batches=1.0, limit_val_batches=1.0, log_every_n_steps=50, log_gpu_memory=None, logger=True, lr=3e-05, max_epochs=None, max_len=1024, max_steps=-1, max_time=None, min_epochs=None, min_steps=None, model_path=None, move_metrics_to_cpu=False, multiple_trainloader_mode='max_size_cycle', num_nodes=1,

In [92]:
#토크나이저 준비
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                       bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                       pad_token='<pad>', mask_token='<mask>') 

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [93]:
# 학습 데이터셋 구축
from sklearn.model_selection import train_test_split
train, dev = train_test_split(df, test_size = 0.2, random_state = seed)

In [94]:
cd data

/content/KoGPT2_summarization/data


In [95]:
train.to_csv('train.tsv', sep='\t')
dev.to_csv('dev.tsv', sep='\t')

In [96]:
cd /content/KoGPT2_summarization

/content/KoGPT2_summarization


In [97]:
from dataset import *

In [98]:
train_dataset = KoGPTSummaryDataset(file='./data/train.tsv',
                                 tok=tokenizer,
                                 max_len=1024,
                                 prompt_length=4)
dev_dataset = KoGPTSummaryDataset(file='./data/dev.tsv',
                                 tok=tokenizer,
                                 max_len=1024,
                                 prompt_length=4)

In [99]:
train_dataloader = DataLoader(train_dataset,
                           batch_size=4,
                           num_workers=2, shuffle=True)

dev_dataloader = DataLoader(dev_dataset,
                           batch_size=4,
                           num_workers=2, shuffle=True)

In [100]:
import argparse
import logging
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers
from torch.utils.data import DataLoader, Dataset
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from soft_embedding import SoftEmbedding
import pdb

In [102]:
# 모델 초기화
model = KoGPTConditionalGeneration(args)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [103]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor='val_loss',
                                                       dirpath='./logs',
                                                       filename='model_chp/{epoch:02d}-{val_loss:.3f}',
                                                       verbose=True,
                                                       save_last=True,
                                                       mode='min',
                                                       save_top_k=3)
tb_logger = pl_loggers.TensorBoardLogger(os.path.join('./logs', 'tb_logs'))
lr_logger = pl.callbacks.LearningRateMonitor()

In [104]:
# trainer 정의
trainer = pl.Trainer.from_argparse_args(args, logger=tb_logger,
                                        callbacks=[checkpoint_callback, lr_logger])

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1585: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`."


In [105]:
trainer.fit(model, train_dataloaders = train_dataloader, val_dataloaders = dev_dataloader)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


NotImplementedError: ignored